## Housing Data Analysis: King County, Washington


#### Authors: Justin Weible, Naomi Weinberger

### Business Problem

In order to help prospective home buyers in the Seattle area, we wanted to be able to predict the price of a house based on various characteristics, including proximity to the downtown Seattle area, year built, number of bedrooms, number of bathrooms, square footage, and the grade it was given by the King County Assessor's Office.

Multiple linear regression techniques were used in order to create a working model that could accurately predict the cost of a house given those specific characteristics.


### Dataset Understanding

The dataset that was used came from King County, Washington, and used housing data of houses from 2014.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df= pd.read_csv('data/kc_house_data.csv')

In [ ]:
df.info()

### Data Cleaning

Before creating our linear regression model, we wanted to remove columns that we didn't think were going to be relevant when analyzing the cost of a house. We eliminated all columns except 'price','sqft_living','bedrooms','bathrooms','floors','grade','condition','yr_built','yr_renovated',and 'waterfront'. 

A new column labeled 'distance" was created, which used a geopy function to use the columns 'lat' and 'long' in order to determine the location of a house based on its distance from downtown. The resulting distance was then converted into miles.

We coverted 'yr_renovated' into a column labeled 'renovated' that would show whether a house had been renovated in its history. The column was converted into 0 for "not renovated" and 1 for "renovated", and all null values were changed to zeroes because we assumed that if a house didn't have a value in that column, it was likely not a renovated house.

We treated the null values in the column 'waterfront' in much the same way. If an entry didn't have a value in waterfront, it was assumed that it was not a waterfront property.

We also created a column called 'age' that took yr_built and subtracted it from 2014 in order to determine the exact age of the house.

A correlation matrix was also used to make sure that our columns were all related to one another in some way.

In [ ]:
from geopy import distance

lat_long = df['lat'].astype(str) + ',' + df['long'].astype(str)
lat_long = list(map(eval, lat_long)) 

Seattle = (47.6062, -122.3321)
miles = []
for i in lat_long:
    miles.append(round(distance.distance(i, Seattle).miles, 1))
df['distance'] = miles

In [ ]:
df=df[['price','sqft_living','bedrooms','bathrooms','floors','grade','condition','yr_built','yr_renovated','waterfront', 'distance']]
df

In [ ]:
df.isnull().sum()

In [ ]:
df['waterfront'] = df['waterfront'].fillna(0)

In [ ]:
df.isnull().sum()
df['yr_renovated'] = df['yr_renovated'].fillna(0)
df.isnull().sum()

In [ ]:
df.rename(columns={'yr_renovated': 'renovated'}, inplace=True)
df.info()

In [ ]:
df['age'] = 2015 - df.yr_built
df = df.drop(columns=['yr_built'], axis=1)

In [ ]:
df.head()

In [ ]:
renovated = df.renovated.values
age = df.age.values
values = np.where(renovated <= 10, 1, 0)
df['renovated'] = np.where(age <= 5, 1, values)

In [ ]:
value_counts_bed=df['bedrooms'].value_counts()
to_remove = value_counts_bed[value_counts_bed <=50].index
df = df[~df.bedrooms.isin(to_remove)]

In [ ]:
value_counts_bath=df['bathrooms'].value_counts()
to_remove_bath = value_counts_bath[value_counts_bath <=80].index
df = df[~df.bathrooms.isin(to_remove_bath)]

In [ ]:
value_counts_floors=df['floors'].value_counts()
to_remove_floor = value_counts_floors[value_counts_floors <=10].index
df = df[~df.floors.isin(to_remove_floor)]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df[['bedrooms', 'bathrooms', 'floors', 'age','renovated','grade','waterfront','sqft_living']].hist(figsize  = [6, 6]); 

In [ ]:
ax = sns.boxplot(x="price", data=df)

In [ ]:
plt.scatter(df['bedrooms'], df['price'], color='green')
plt.title('Bedrooms and Price', fontsize=14)
plt.xlabel('Number of Bedrooms', fontsize=14)
plt.ylabel('House Price', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
sns.jointplot(data=df, x="bedrooms", y="price", kind="reg")

In [ ]:
sns.jointplot(data=df, x="grade", y="price", kind="reg")

In [ ]:
sns.jointplot(data=df, x="bathrooms", y="price", kind="reg")

In [ ]:
sns.jointplot(data=df, x="floors", y="price", kind="reg")

In [ ]:
sns.jointplot(data=df, x="age", y="price", kind="reg")

In [ ]:
sns.jointplot(data=df, x="renovated", y="price", kind="reg")

In [ ]:
plt.scatter(df['bathrooms'], df['price'], color='blue')
plt.title('Bathrooms and Price', fontsize=14)
plt.xlabel('Number of Bathrooms', fontsize=14)
plt.ylabel('House Price', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(df['floors'], df['price'], color='red')
plt.title('Floors and Price', fontsize=14)
plt.xlabel('Number of Floors', fontsize=14)
plt.ylabel('House Price', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
plt.scatter(df['age'], df['price'], color='orange')
plt.title('House Age and Price', fontsize=14)
plt.xlabel('Age', fontsize=14)
plt.ylabel('House Price', fontsize=14)
plt.grid(True)
plt.show()

In [ ]:
corr=df.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

## Model 1

For our first model, we simply took the dataset and its columns and created the initial model based on the price of a house just to give us a starting point.

The first model had an R-squared value of 0.692 with a Test RMSE of 185212.7798740588 and a Train RMSE of 183591.80606517757.

Based on the QQ-plot and the distribution of the column 'price', we determined that there were significant outliers just in price that would need to be accounted for in our second model.

In [ ]:
def tts_model(df):
    y = df['price']
    X = df.drop(columns=['price'], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=123)
    
    linreg = LinearRegression()
    model = linreg.fit(X_train, y_train)

    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    mse_train = mean_squared_error(y_train, train_predictions)
    mse_test = mean_squared_error(y_test, test_predictions)
    
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    print('Test RMSE:', np.sqrt(mse_test))
    print('Train RMSE:', np.sqrt(mse_train))
    print('Training Score:', round(train_score, 2))
    print('Test Score:', round(test_score, 2))
    print('Coefficients:', model.coef_)

    X_train = sm.add_constant(X_train)
    smmodel = sm.OLS(y_train, X_train).fit()
    print(smmodel.summary())
    return smmodel

In [ ]:
model = tts_model(df)

In [ ]:
plt.figure(figsize=(12, 8))
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)
plt.title('QQ Plot')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(df.price)
plt.title('Price Distribution')
plt.tight_layout()

## Model 2 (Normalizing Price)

Our second model was created just to remove outliers in price. A figure of $1,000,000 was chosen as the maximum price of a house to limit the prediction model to what we consider to be an affordable cost. It also gave the data in price a much more normalized distribution.

The R-squared value dropped to 0.636, but we saw a significant improvement in our Test RMSE (Root Mean Square Error) and our Train RMSE.

In [ ]:
df = df[df['price'] <= 1000001]

In [ ]:
plt.figure(figsize=(12, 8))
sns.distplot(df.price)
plt.title('Price Distribution')
plt.tight_layout()

In [ ]:
ax = sns.boxplot(x="price", data=df)

In [ ]:
df.info()

In [ ]:
model = tts_model(df)

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)
plt.title('QQ Plot')
plt.show()

## Model 3 (Multicollinearity)

Our third model took multicollinearity into account. Looking at correlation pairs, the only strong correlation between two independent variables was between sqft_living and bathrooms. But because none of the variance inflation factors were higher than 3.3, multicollinearity didn't appear to be a real issue in this model.

In [ ]:
X = df.drop(columns=['price'], axis=1)
plt.figure(figsize=(14, 10))
sns.heatmap(X.corr(), center=0)
plt.tight_layout()
plt.show()

In [ ]:
abs(X.corr()) >= .7

In [ ]:
corr_df = X.corr().abs().stack().reset_index().sort_values(0, ascending=False)

# Zip the variable name columns in a  column named "pairs"
corr_df['pairs'] = list(zip(corr_df.level_0, corr_df.level_1))

# Set index to pairs
corr_df.set_index(['pairs'], inplace = True)

# Drop level columns
corr_df.drop(columns=['level_1', 'level_0'], inplace = True)

# Rename correlation column to cc
corr_df.columns = ['cc']

# Drop duplicates
corr_df.drop_duplicates(inplace=True)

In [ ]:
corr_df[(corr_df.cc>.70) & (corr_df.cc <1)]

In [ ]:
X = sm.add_constant(X)
vif = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_dict = dict(zip(X.columns, vif))
vif_dict

In [ ]:
tts_model(df)

## Model 4 (Removing Insignificant Features)

Based on the p-values in our model, no feature was considered statistically insignificant. However, we decided to drop 'renovated' from our model just to see if it improved the accuracy of our model.

Removing 'renovated' did improve the Test RMSE and the Train RMSE slightly, although the R-squared value did decrease.

In [ ]:
df = df.drop(columns=['renovated'], axis=1)

In [ ]:
tts_model(df)

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)
plt.title('QQ Plot')
plt.show()

## Model 5: Adjusting for Outliers

For Model 5, we wanted to look at outliers in the multiple independent variables to see if that was affecting our model in some way.

Through looking at multiple graphs, the biggest outliers were in 'sqft_living', 'bathrooms', and 'floors'.

Based on those graphs, we decided to limit our model to evaluating houses that were less that 5,000 square feet, had fewer that 4 bathrooms, and had 3 floors or less.

Refining the model in this way, our RMSE actually increased and our R-squared increased to 0.636. 

However, the difference between our Train RMSE and Test RMSE narrowed from previous models, and a higher Test RMSE suggests that the data in our model might be slightly overfitted.

In [ ]:
#View distribution plots for all columns
for col in df.columns:
    plt.subplots(1, 1)
    sns.distplot(df[col])

In [ ]:
df = df[df['sqft_living'] < 5000]
df = df[df['bathrooms'] < 4.0]
df = df[df['floors'] < 3.5]

In [ ]:
tts_model(df)

In [ ]:
fig = sm.graphics.qqplot(model.resid, dist=stats.norm, line='45', fit=True)
plt.title('QQ Plot')
plt.show()

## Homoscedascity

Model 5 became our final model, so we did a quick check for homoscedascity. With the exception of a few outliers, our residuals are relatively homoscedastic.

In [ ]:
y = df['price']
X = df.drop(columns=['price'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=123)
    
linreg = LinearRegression()
model = linreg.fit(X_train, y_train)

train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

mse_train = mean_squared_error(y_train, train_predictions)
mse_test = mean_squared_error(y_test, test_predictions)
    
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)

print('Test RMSE:', np.sqrt(mse_test))
print('Train RMSE:', np.sqrt(mse_train))
print('Training Score:', round(train_score, 2))
print('Test Score:', round(test_score, 2))
print('Coefficients:', model.coef_)

X_train = sm.add_constant(X_train)
smmodel = sm.OLS(y_train, X_train).fit()
print(smmodel.summary())

In [ ]:
plt.scatter(train_predictions, smmodel.resid)

## Plots

In [ ]:
plt.figure(figsize=(12, 8))
sns.barplot(df['distance'], df['price'])
plt.title('Price vs. Distance', fontsize=16)
plt.xlabel('Distance')
plt.ylabel('Price')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(df.grade, df.age)
plt.title('Building Grade vs. Age of Home', fontsize=16)
plt.xlabel('Grade')
plt.ylabel('Age')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(df.grade, df.price)
plt.title('Building Grade vs. Sale Price', fontsize=16)
plt.xlabel('Grade')
plt.ylabel('Price')
plt.tight_layout()
plt.show()

In [ ]:
bedrooms_1_5 = df.loc[(df['distance'] <= 5) & (df['bedrooms'] < 2)]
bedrooms_2_5 = df.loc[(df['distance'] <= 5) & (df['bedrooms'] >= 2) & (df['bedrooms'] < 3)]
bedrooms_3_5 = df.loc[(df['distance'] <= 5) & (df['bedrooms'] >= 3) & (df['bedrooms'] < 4)]
bedrooms_4_5 = df.loc[(df['distance'] <= 5) & (df['bedrooms'] == 4)]

In [ ]:
bedrooms_1 = df.loc[(df['distance'] > 5) & (df['bedrooms'] < 2)]
bedrooms_2 = df.loc[(df['distance'] > 5) & (df['bedrooms'] >= 2) & (df['bedrooms'] < 3)]
bedrooms_3 = df.loc[(df['distance'] > 5) & (df['bedrooms'] >= 3) & (df['bedrooms'] < 4)]
bedrooms_4 = df.loc[(df['distance'] > 5) & (df['bedrooms'] == 4)]

In [ ]:
names = ['1 Bedroom Houses', '2 Bedroom Houses', '3 Bedroom Houses', '4 Bedroom Houses']
medians_5 = [bedrooms_1_5.price.median(), bedrooms_2_5.price.median(), bedrooms_3_5.price.median(), bedrooms_4_5.price.median()]
medians = [bedrooms_1.price.median(), bedrooms_2.price.median(), bedrooms_3.price.median(), bedrooms_4.price.median()]

In [ ]:
sns.set_palette('Reds_r')
plt.figure(figsize=(13, 6))
plt.subplot(1, 2, 1)
sns.barplot(medians_5, names)
for i in range(len(medians_5)):
    plt.text(x= (medians_5[i]/2) - 50000, y=i, s=int(medians_5[i]), c='white', fontsize=14)
plt.title('Less Than 5 Miles to Downtown', fontsize=16)
plt.xlabel('Price')
plt.yticks(fontsize=14)

plt.subplot(1, 2, 2)
for i in range(len(medians)):
    plt.text(x= (medians[i]/2) - 50000, y=i, s=int(medians[i]), c='white', fontsize=14)
sns.barplot(medians, names)
plt.title('More Than 5 Miles from Downtown', fontsize=16)
plt.xlabel('Price')
plt.yticks(fontsize=14)
plt.subplots_adjust(wspace=0.4)
plt.tight_layout()
plt.show()

In [ ]:
sqft_under1250_5 = df.loc[(df['distance'] <= 5) & (df['sqft_living'] < 1250)]
sqft_under2500_5 = df.loc[(df['distance'] <= 5) & (df['sqft_living'] >= 1250) & (df['sqft_living'] < 2500)]
sqft_under3750_5 = df.loc[(df['distance'] <= 5) & (df['sqft_living'] >= 2500) & (df['sqft_living'] < 3750)]
sqft_under5000_5 = df.loc[(df['distance'] <= 5) & (df['sqft_living'] >= 3750) & (df['sqft_living'] <= 5000)] 

In [ ]:
sqft_under1250 = df.loc[(df['distance'] > 5) & (df['sqft_living'] < 1250)]
sqft_under2500 = df.loc[(df['distance'] > 5) & (df['sqft_living'] >= 1250) & (df['sqft_living'] < 2500)]
sqft_under3750 = df.loc[(df['distance'] > 5) & (df['sqft_living'] >= 2500) & (df['sqft_living'] < 3750)]
sqft_under5000 = df.loc[(df['distance'] > 5) & (df['sqft_living'] >= 3750) & (df['sqft_living'] <= 5000)] 

In [ ]:
names = ['Under 1250 Square Feet', 'Under 2500 Square Feet', 'Under 3250 Square Feet', 'Under 5000 Square Feet']
sqft_medians_5 = [sqft_under1250_5.price.median(), sqft_under2500_5.price.median(), sqft_under3750_5.price.median(), sqft_under5000_5.price.median()]
sqft_medians = [sqft_under1250.price.median(), sqft_under2500.price.median(), sqft_under3750.price.median(), sqft_under5000.price.median()]

In [ ]:
sns.set_palette('Reds_r')
plt.figure(figsize=(13, 6))
plt.subplot(1, 2, 1)
sns.barplot(sqft_medians_5, names)
for i in range(len(sqft_medians_5)):
    plt.text(x= (sqft_medians_5[i]/2) - 50000, y=i, s=int(sqft_medians_5[i]), c='white', fontsize=14)
plt.title('Less Than 5 Miles to Downtown', fontsize=16)
plt.xlabel('Price')
plt.yticks(fontsize=14)

plt.subplot(1, 2, 2)
for i in range(len(sqft_medians)):
    plt.text(x= (sqft_medians[i]/2) - 50000, y=i, s=int(sqft_medians[i]), c='white', fontsize=14)
sns.barplot(sqft_medians, names)
plt.title('More Than 5 Miles from Downtown', fontsize=16)
plt.xlabel('Square Feet')
plt.yticks(fontsize=14)
plt.subplots_adjust(wspace=0.4)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(df.bathrooms, df.price)
plt.title('Bathrooms vs. Sale Price', fontsize=16)
plt.xlabel('Bathrooms')
plt.ylabel('Price')
plt.tight_layout()
plt.show()

### Final Conclusions

* Location is everything. Houses that are located closer to downtown are generally more expensive than houses that are located outside of downtown.

* If you're on a budget and looking to buy a small house, a house outside of the city should be cheaper than one inside the city.

* Larger family homes also might be cheaper outside of the city. However, it appears that some of the data on the cost of larger houses might be affected by some houses that are listed as waterfront property.

* Generally speaking, the age of a home also affects its price. According to the King County Assessor's grading system, older homes tend to have lower grades compared to newer ones. Building grade also affects price, as the higher the building grade, the higher the price.

* Bathrooms also tend to affect the cost of a house. Houses with more bathrooms tend to be priced higher than houses with fewer bathrooms.

## Recommendations

* Linear regression doesn't appear to be the best method of predicting housing prices with great accuracy. An error of over $100,000 in price and some outliers in the residuals seems to show that. Other types of models should be considered in order to see if a different type of model might be more accurate.